In [2]:
from dwca.read import DwCAReader
from dwca.darwincore.utils import qualname as qn
#from pygbif import maps
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import branca

with DwCAReader('data/SFSU_DwC-A.zip') as dwca:
    print ("Core data file is: {}".format(dwca.descriptor.core.file_location))  

    
    df = dwca.pd_read('occurrences.csv', parse_dates=True, low_memory=False)


#print(df.columns)


MGP = df[df["locality"].str.contains('Audubon Canyon Ranch', na=False)]
MGP_dropna = MGP[MGP["decimalLatitude"].notna()]
MGP_dropna = MGP_dropna[MGP_dropna["decimalLongitude"].notna()]





obs = MGP_dropna[["decimalLatitude", "decimalLongitude","scientificName", "habitat", "locality", "eventDate", "recordedBy"]].reset_index()



def popup_html(row):
    
    i = row
    scientificName=obs['scientificName'].iloc[i] 
    habitat=obs['habitat'].iloc[i]
    locality = obs['locality'].iloc[i] 
    obsDate=obs['eventDate'].iloc[i] 
    recordedBy = obs['recordedBy'].iloc[i] 

    left_col_color = "#19a7bd"
    right_col_color = "#f2f0d3"
    
    html = """<!DOCTYPE html>
<html>
<head>
<h4 style="margin-bottom:10"; width="200px"; font-size:30px>{}</h4>""".format(scientificName) + """
</head>
    <table style="height: 126px; width: 350px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Habitat</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(habitat) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Locality</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(locality) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Observation Date</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(obsDate) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Recorded By</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(recordedBy) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

obs_mean_location = obs.decimalLatitude.mean(), obs.decimalLongitude.mean()

myMap = folium.Map(location = obs_mean_location, zoom_start=14.5, control_scale=True, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri')

print(len(obs))

markercluster = MarkerCluster(control=False).add_to(myMap)

for i in range(0, len(obs)):

    html = popup_html(i)
    iframe = branca.element.IFrame(html=html, width=510, height=280)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)

    folium.Marker([obs["decimalLatitude"].iloc[i], obs["decimalLongitude"].iloc[i]],
    popup=popup).add_to(markercluster)

myMap.save("map.html")
myMap





Core data file is: occurrences.csv
468
